Test MySQL

In [51]:
import sys
import os
import json
import logging
from kafka import KafkaConsumer, KafkaProducer
from json import loads
import random
from datetime import datetime
import mysql.connector
from logging.handlers import RotatingFileHandler
from time import sleep

In [ ]:
def mysql_consumer2():
    mysql_conn = mysql.connector.connect(
        host='localhost',
        user='mysql',
        password='mysql',
        database='oaken'
    )
    mysql_cursor = mysql_conn.cursor()

    # Create a consumer instance
    consumer = KafkaConsumer(
        'mysql',
        bootstrap_servers=['localhost:19092'],
        auto_offset_reset='earliest',  # Start consuming from the earliest offset
        enable_auto_commit=True,       # Automatically commit offsets
        group_id='oaken_mysql_group',  # Specify a consumer group
        value_deserializer=lambda x: loads(x.decode('utf-8')))

    consumer.subscribe(topics=['mysql'])

    invoice_producer = KafkaProducer(bootstrap_servers=['localhost:19092'],
                         value_serializer=lambda x: json.dumps(x).encode('utf-8'))


    # Poll for messages
    try:
        for message in consumer:
            try:
                data = message.value
                # Customer
                storNumber = int(data.get('StoreNumber', ''))
                if not storNumber:
                    pass

                storeName = data.get('StoreName', '')
                address = data.get('Address', '')
                city = data.get('City', '')
                county = data.get('County', '')
                state = data.get('State', '')
                zip_code = int(data.get('ZipCode', ''))

                # vendor
                vendorNumber = int(float(data.get('VendorNumber', '')))
                if not vendorNumber:
                    pass

                vendorName = data.get('VendorName', '')

                # category
                category = int(float(data.get('Category','')))
                if not category:
                    pass

                categoryName = (data.get('CategoryName',''))

                # product
                itemNumber = int(data.get('ItemNumber', ''))
                if not itemNumber:
                    pass
                itemDescription = data.get('ItemDescription', '')
                pack = int(data.get('Pack', ''))
                volume = int(data.get('BottleVolumeML', ''))
                cost = float(data.get('BottleCost', '').replace('$', ''))
                retail = float(data.get('BottleRetail', '').replace('$', ''))

                # Sales
                invoice = data.get('Invoice', '')
                if not invoice:
                    pass

                date_str = data.get('Date', '')
                if not date_str:
                    pass

                sales_date = datetime.datetime.strptime(date_str, '%m/%d/%Y').date()

                amountSold = int(data.get('BottlesSold', ''))
                totalLiters = float(data.get('VolumeSoldLiters', ''))

                sales_str = data.get('SaleDollars', '').replace('$', '')
                sales = float(sales_str)

                # MySQL
                '''
                Multiple try/except blocks are used due to the simplistic invoicing
                application script.

                In a real world scenario, likely each of these blocks would be done
                as a truly separate process.

                The try/except prevents the duplicates failing to be added to the
                database from preventing the rest of the processes from competing.
                '''

                print('customer')
                try:
                    CUSTOMER_QUERY = '''
                        INSERT INTO customer (StoreNumber,StoreName,Address,City,CountyName,State,ZipCode)
                        VALUES (%s,%s,%s,%s,%s,%s,%s)
                        '''
                    customer_data = (storNumber,storeName,address,city,county,state,zip_code)
                    mysql_cursor.execute(CUSTOMER_QUERY, customer_data)
                    mysql_conn.commit()
                except Exception as e:
                    pass

                print('vendor')
                try:
                    VENDOR_QUERY = '''
                        INSERT INTO vendor (VendorNumber,VendorName)
                        VALUES (%s,%s)
                        '''
                    vendor_data = (vendorNumber,vendorName)
                    mysql_cursor.execute(VENDOR_QUERY,vendor_data)
                    mysql_conn.commit()
                except Exception as e:
                    pass

                print(category)
                try:
                    CATEGORY_QUERY = '''
                        INSERT INTO category (CategoryNumber,CategoryName)
                        VALUES (%s,%s)
                        '''
                    category_data = (category,categoryName)
                    mysql_cursor.execute(CATEGORY_QUERY, category_data)
                    mysql_conn.commit()
                except Exception as e:
                    pass

                print('product')
                try:
                    PRODUCT_QUERY = '''
                        INSERT INTO product (ItemNumber,CategoryNumber,ItemDescription,BottleVolumeML,
                        Pack,BottleCost,BottleRetail)
                        VALUES (%s,%s,%s,%s,%s,%s,%s)
                        '''
                    product_data = (itemNumber,category,itemDescription,volume,pack,cost,retail)
                    mysql_cursor.execute(PRODUCT_QUERY, product_data)
                    mysql_conn.commit()
                except Exception as e:
                    pass

                print(sales)
                try:
                    SALES_QUERY = '''
                    INSERT INTO sales (Invoice,StoreNumber,VendorNumber,SaleDate,SaleDollars,
                    ItemNumber,VolumeSoldLiters,BottlesSold)
                    VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
                    '''
                    sales_data = (invoice,storNumber,vendorNumber,sales_date,sales,itemNumber,
                                    totalLiters,amountSold)
                    mysql_cursor.execute(SALES_QUERY, sales_data)
                    mysql_conn.commit()
                except Exception as e:
                    pass

                print('topic')
                # Topic should post after MySQL processing to ensure data is in the database.
                try:
                    sales_date_str = str(sales_date)
                    INVOICES_info = {
                        "Invoice": invoice,
                        "SaleDate": date_str,
                        "saleDollars": sales_str
                    }

                    invoice_producer.send('invoices', value=INVOICES_info)
                    invoice_producer.flush()
                except Exception as e:
                    pass

            except Exception as e:
                pass

    # Close the consumer
    finally:
        consumer.close()
        mysql_conn.close()

In [46]:
def mysql_consumer():
    mysql_conn = mysql.connector.connect(
        host='localhost',
        user='mysql',
        password='mysql',
        database='oaken'
    )
    mysql_cursor = mysql_conn.cursor()

    # Create a consumer instance
    consumer = KafkaConsumer(
        'mysql',
        bootstrap_servers=['localhost:9092'],
        auto_offset_reset='earliest',  # Start consuming from the earliest offset
        enable_auto_commit=True,       # Automatically commit offsets
        group_id='my_consumer_group',  # Specify a consumer group
        value_deserializer=lambda x: loads(x.decode('utf-8')))

    consumer.subscribe(topics=['mysql'])

    invoice_producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         value_serializer=lambda x: json.dumps(x).encode('utf-8'))


    # Poll for messages
    try:
        for message in consumer:
            try:
                data = message.value
                # Customer
                storNumber = int(data.get('StoreNumber', ''))
                if not storNumber:
                    print("StoreNumber is null or invalid. Skipping insertion.")
                    pass

                storeName = data.get('StoreName', '')
                address = data.get('Address', '')
                city = data.get('City', '')
                county = data.get('County', '')
                state = data.get('State', '')
                zip_code = int(data.get('ZipCode', ''))

                # vendor
                vendorNumber = int(float(data.get('VendorNumber', '')))
                if not vendorNumber:
                    print("VendorNumber is null or invalid. Skipping insertion.")
                    pass

                vendorName = data.get('VendorName', '')

                # category
                category = int(float(data.get('Category','')))
                if not category:
                    print("Category is null or invalid. Skipping insertion.")
                    pass

                categoryName = (data.get('CategoryName',''))

                # product
                itemNumber = int(data.get('ItemNumber', ''))
                if not itemNumber:
                    print("ItemNumber is null or invalid. Skipping insertion.")
                    pass
                itemDescription = data.get('ItemDescription', '')
                pack = int(data.get('Pack', ''))
                volume = int(data.get('BottleVolumeML', ''))
                cost = float(data.get('BottleCost', '').replace('$', ''))
                retail = float(data.get('BottleRetail', '').replace('$', ''))

                # Sales
                invoice = data.get('Invoice', '')
                if not invoice:
                    print("Invoice is null or invalid. Skipping insertion.")
                    pass

                date_str = data.get('Date', '')
                if not date_str:
                    print("Date is null or invalid. Skipping insertion.")
                    pass

                sales_date = datetime.strptime(date_str, '%m/%d/%Y').date()

                amountSold = int(data.get('BottlesSold', ''))
                totalLiters = float(data.get('VolumeSoldLiters', ''))

                sales_str = data.get('SaleDollars', '').replace('$', '')
                sales = float(sales_str)

                # MySQL
                '''
                Multiple try/except blocks are used due to the simplistic invoicing
                application script.

                In a real world scenario, likely each of these blocks would be done
                as a truly separate process.

                The try/except prevents the duplicates failing to be added to the
                database from preventing the rest of the processes from competing.
                '''

                print('customer')
                try:
                    CUSTOMER_QUERY = '''
                        INSERT INTO customer (StoreNumber,StoreName,Address,City,CountyName,State,ZipCode)
                        VALUES (%s,%s,%s,%s,%s,%s,%s)
                        '''
                    customer_data = (storNumber,storeName,address,city,county,state,zip_code)
                    mysql_cursor.execute(CUSTOMER_QUERY, customer_data)
                    mysql_conn.commit()
                except Exception as e:
                    print(f"Error processing message: {e}")
                    pass

                print('vendor')
                try:
                    VENDOR_QUERY = '''
                        INSERT INTO vendor (VendorNumber,VendorName)
                        VALUES (%s,%s)
                        '''
                    vendor_data = (vendorNumber,vendorName)
                    mysql_cursor.execute(VENDOR_QUERY,vendor_data)
                    mysql_conn.commit()
                except Exception as e:
                    print(f"Error processing message: {e}")
                    pass

                print(category)
                try:
                    CATEGORY_QUERY = '''
                        INSERT INTO category (CategoryNumber,CategoryName)
                        VALUES (%s,%s)
                        '''
                    category_data = (category,categoryName)
                    mysql_cursor.execute(CATEGORY_QUERY, category_data)
                    mysql_conn.commit()
                except Exception as e:
                    print(f"Error processing message: {e}")
                    pass

                print('product')
                try:
                    PRODUCT_QUERY = '''
                        INSERT INTO product (ItemNumber,CategoryNumber,ItemDescription,BottleVolumeML,
                        Pack,BottleCost,BottleRetail)
                        VALUES (%s,%s,%s,%s,%s,%s,%s)
                        '''
                    product_data = (itemNumber,category,itemDescription,volume,pack,cost,retail)
                    mysql_cursor.execute(PRODUCT_QUERY, product_data)
                    mysql_conn.commit()
                except Exception as e:
                    print(f"Error processing message: {e}")
                    pass

                print(sales)
                try:
                    SALES_QUERY = '''
                    INSERT INTO sales (Invoice,StoreNumber,VendorNumber,SaleDate,SaleDollars,
                    ItemNumber,VolumeSoldLiters,BottlesSold)
                    VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
                    '''
                    sales_data = (invoice,storNumber,vendorNumber,sales_date,sales,itemNumber,
                                    totalLiters,amountSold)
                    mysql_cursor.execute(SALES_QUERY, sales_data)
                    mysql_conn.commit()
                except Exception as e:
                    print(f"Error processing message: {e}")
                    pass

                print('topic')
                # Topic should post after MySQL processing to ensure data is in the database.
                try:
                    sales_date_str = str(sales_date)
                    INVOICES_info = {
                        "Invoice": invoice,
                        "SaleDate": sales_date_str,
                        "saleDollars": sales_str
                    }

                    invoice_producer.send('invoices', value=INVOICES_info)
                    invoice_producer.flush()
                except Exception as e:
                    print(f"Error processing message: {e}")
                    pass

            except Exception as e:
                print(f"Error processing message: {e}")
                pass

    # Close MySQL connection
    finally:
        mysql_cursor.close()
        mysql_conn.close()
        sleep(1)

In [49]:
mysql_consumer()

customer
vendor
1031080
product
166.68
topic
customer
vendor
1011300
product
45.22
topic
customer
vendor
1062200
product
509.04
topic
customer
vendor
1031080
Error processing message: 1062 (23000): Duplicate entry '1031080' for key 'category.PRIMARY'
product
20.76
topic
customer
vendor
1081300
product
69.36
topic
customer
vendor
1031080
Error processing message: 1062 (23000): Duplicate entry '1031080' for key 'category.PRIMARY'
product
166.44
topic
customer
vendor
1052010
product
62.5
topic
customer
vendor
1082900
product
34.96
topic
customer
vendor
1032080
product
119.52
topic
customer
vendor
Error processing message: 1062 (23000): Duplicate entry '260' for key 'vendor.PRIMARY'
1012100
product
959.4
topic
customer
vendor
Error processing message: 1062 (23000): Duplicate entry '35' for key 'vendor.PRIMARY'
1032080
Error processing message: 1062 (23000): Duplicate entry '1032080' for key 'category.PRIMARY'
product
323.52
topic
customer
vendor
1071100
product
13.86
topic
customer
vendor


KeyboardInterrupt: 

In [ ]:
KAFKA_SERVER='localhost:9092'

MYSQL_HOST='localhost'
MYSQL_ROOT_PASSWORD='mysql'
MYSQL_USER='mysql'
MYSQL_PASSWORD='mysql'
MYSQL_DATABASE='oaken'


def mysql_consumer2():

    # MySQL connection
    mysql_conn = mysql.connector.connect(
        host=MYSQL_HOST,
        user=MYSQL_USER,
        password=MYSQL_PASSWORD,
        database=MYSQL_DATABASE
    )

    mysql_cursor = mysql_conn.cursor()

    # Create a consumer instance
    consumer = KafkaConsumer(
        'mysql',
        bootstrap_servers=[KAFKA_SERVER],
        auto_offset_reset='earliest',  # Start consuming from the earliest offset
        enable_auto_commit=True,       # Automatically commit offsets
        group_id='oaken_mysql_group',  # Specify a consumer group
        value_deserializer=lambda x: loads(x.decode('utf-8')))

    consumer.subscribe(topics=['mysql'])

    invoice_producer = KafkaProducer(bootstrap_servers=[KAFKA_SERVER],
                            value_serializer=lambda x: json.dumps(x).encode('utf-8'))

    print('set up complete')
    # Poll for messages
    try:
        for message in consumer:
            try:
                print('begin reading')
                data = message.value
                # Customer
                storNumber = int(data.get('StoreNumber', ''))
                if not storNumber:
                    print('storeNumber error')
                    pass

                storeName = data.get('StoreName', '')
                address = data.get('Address', '')
                city = data.get('City', '')
                county = data.get('County', '')
                state = data.get('State', '')
                zip_code = int(data.get('ZipCode', ''))

                # vendor
                vendorNumber = int(float(data.get('VendorNumber', '')))
                if not vendorNumber:
                    pass

                vendorName = data.get('VendorName', '')

                # category
                category = int(float(data.get('Category','')))
                if not category:
                    pass

                categoryName = (data.get('CategoryName',''))

                # product
                itemNumber = int(data.get('ItemNumber', ''))
                if not itemNumber:
                    pass
                itemDescription = data.get('ItemDescription', '')
                pack = int(data.get('Pack', ''))
                volume = int(data.get('BottleVolumeML', ''))
                cost = float(data.get('BottleCost', '').replace('$', ''))
                retail = float(data.get('BottleRetail', '').replace('$', ''))

                # Sales
                invoice = data.get('Invoice', '')
                if not invoice:
                    pass

                date_str = data.get('Date', '')
                if not date_str:
                    pass

                sales_date = datetime.datetime.strptime(date_str, '%m/%d/%Y').date()

                amountSold = int(data.get('BottlesSold', ''))
                totalLiters = float(data.get('VolumeSoldLiters', ''))

                sales_str = data.get('SaleDollars', '').replace('$', '')
                sales = float(sales_str)

                # MySQL
                '''
                Multiple try/except blocks are used due to the simplistic invoicing
                application script.

                In a real world scenario, likely each of these blocks would be done
                as a truly separate process.

                The try/except prevents the duplicates failing to be added to the
                database from preventing the rest of the processes from competing.
                '''

                print('begin queries')
                try:
                    CUSTOMER_QUERY = '''
                        INSERT INTO customer (StoreNumber,StoreName,Address,City,CountyName,State,ZipCode)
                        VALUES (%s,%s,%s,%s,%s,%s,%s)
                        '''
                    customer_data = (storNumber,storeName,address,city,county,state,zip_code)
                    mysql_cursor.execute(CUSTOMER_QUERY, customer_data)
                    mysql_conn.commit()
                except Exception as e:
                    print(f"Error processing message: {e}")
                    pass

                try:
                    VENDOR_QUERY = '''
                        INSERT INTO vendor (VendorNumber,VendorName)
                        VALUES (%s,%s)
                        '''
                    vendor_data = (vendorNumber,vendorName)
                    mysql_cursor.execute(VENDOR_QUERY,vendor_data)
                    mysql_conn.commit()
                except Exception as e:
                    print(f"Error processing message: {e}")
                    pass

                try:
                    CATEGORY_QUERY = '''
                        INSERT INTO category (CategoryNumber,CategoryName)
                        VALUES (%s,%s)
                        '''
                    category_data = (category,categoryName)
                    mysql_cursor.execute(CATEGORY_QUERY, category_data)
                    mysql_conn.commit()
                except Exception as e:
                    print(f"Error processing message: {e}")
                    pass

                try:
                    PRODUCT_QUERY = '''
                        INSERT INTO product (ItemNumber,CategoryNumber,ItemDescription,BottleVolumeML,
                        Pack,BottleCost,BottleRetail)
                        VALUES (%s,%s,%s,%s,%s,%s,%s)
                        '''
                    product_data = (itemNumber,category,itemDescription,volume,pack,cost,retail)
                    mysql_cursor.execute(PRODUCT_QUERY, product_data)
                    mysql_conn.commit()
                except Exception as e:
                    print(f"Error processing message: {e}")
                    pass

                try:
                    SALES_QUERY = '''
                    INSERT INTO sales (Invoice,StoreNumber,VendorNumber,SaleDate,SaleDollars,
                    ItemNumber,VolumeSoldLiters,BottlesSold)
                    VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
                    '''
                    sales_data = (invoice,storNumber,vendorNumber,sales_date,sales,itemNumber,
                                    totalLiters,amountSold)
                    mysql_cursor.execute(SALES_QUERY, sales_data)
                    mysql_conn.commit()
                except Exception as e:
                    print(f"Error processing message: {e}")
                    pass

                # Topic should post after MySQL processing to ensure data is in the database.
                try:
                    sales_date_str = str(sales_date)
                    INVOICES_info = {
                        "Invoice": invoice,
                        "SaleDate": date_str,
                        "saleDollars": sales_str
                    }

                    invoice_producer.send('invoices', value=INVOICES_info)
                    invoice_producer.flush()
                except Exception as e:
                    print(f"Error processing message: {e}")
                    pass

            except Exception as e:
                print(f"Error processing message: {e}")
                pass

        # Close the consumer
    finally:
        consumer.close()
        mysql_conn.close()

In [ ]:
mysql_consumer2()

In [ ]:
import pandas as pd

df = pd.read_csv("data/Iowa_Liquor_Sales.csv", nrows=10)

print(df)

In [ ]:
print(df.columns)

In [ ]:
# sample output
'''
{'Invoice/Item Number': 'S29198800001', 'Date': '11/20/2015', 'Store Number': '2191', 'Store Name': 'Keokuk Spirits', 
'Address': '1013 MAIN', 'City': 'KEOKUK', 'Zip Code': '52632', 'Store Location': '1013 MAIN\nKEOKUK 52632\n(40.39978, -91.387531)', 
'County Number': '56', 'County': 'Lee', 'Category': '', 'Category Name': '', 'Vendor Number': '255', 'Vendor Name': 'Wilson Daniels Ltd.', 
'Item Number': '297', 'Item Description': 'Templeton Rye w/Flask', 'Pack': '6', 'Bottle Volume (ml)': '750', 'State Bottle Cost': '$18.09', 
'State Bottle Retail': '$27.14', 'Bottles Sold': '6', 'Sale (Dollars)': '$162.84', 'Volume Sold (Liters)': '4.50', 'Volume Sold (Gallons)': '1.19'}
'''

In [ ]:
with open("../data/Iowa_Liquor_Sales.csv", 'r') as file:
    reader = csv.DictReader(file)
    while True:
        chunk = []
        for _ in range(1000):
            try:
                chunk.append(next(reader))
            except StopIteration:
                break
        if not chunk:
            break
        for row in chunk:
            print(row)

In [ ]:
# money conversion test
data = {'SaleDollars': '$12.34'}

sales = float(data.get('SaleDollars','').replace('$', ''))

sales

In [ ]:
# datetime test
from datetime import datetime

date_data = {"Date":"11/09/2015"}

date_string = date_data.get('Date', '')
invoice_date = datetime.strptime(date_string,'%m/%d/%Y').date()

print(invoice_date)
